In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM
)
import json, csv
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import pickle
from tqdm import trange,tqdm

In [ ]:
import pandas as pd
data = pd.read_csv('D:\\download\\比賽\\claim_evidence.csv',keep_default_na=False,na_values=[''])
data_11620 = pd.read_csv('D:\\download\\比賽\\claim_evidence_train_all.csv',keep_default_na=False,na_values=['']) 
wiki_data = pd.read_csv('D:\\download\\比賽\\wiki_clean_doc.csv', keep_default_na=False, na_values=[' '], index_col=False)

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import random

# 用evidence做fine-tuned

In [ ]:
train_examples = []
#for i in tqdm(range(0,20)):
for i in tqdm(range(len(data_11620))):
    for j in eval(data['text'][i]):
        tmp = [data['claim'][i],j]
        print(tmp)
        train_examples.append(InputExample(texts=tmp, label=1.0))

In [ ]:
#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('hfl/chinese-roberta-wwm-ext-large', device='cuda')
#Define your train examples. You need more than just two examples...
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5000, warmup_steps=1000)

In [ ]:
model.save('D:\\Vic\\GitHub\\Team3068_FactualTextRetrieval\\sbert\\model\\hfl_pretraineds_0511sentBase')

# 用evidence的article做fine-tuned

In [ ]:
def check_article(item):
    tmp = []
    for i in item :
        if isinstance(i[0], list) == True:
            for j in i:
                if j[2] == '臺灣海峽危機#第二次臺灣海峽危機（1958）':
                    tmp = []
                else:
                    tmp += [[j[2]]]
    return tmp

In [ ]:
train_examples = []
tmp_ = []
train = []
for i in tqdm(range(len(data))):
    article = check_article(eval(data['evidence'][i]))
    if len(article) != 0 :
        for j in article:
            tmp = [data['claim'][i]]
            tmp += [j[0]]
            if tmp in tmp_:
                continue
            else:
                #print(tmp)
                tmp_ += [tmp]
                train_examples.append(InputExample(texts=tmp, label=1.0))

In [ ]:
with open('D:\\download\\比賽\\train_examples.pickle', 'rb') as f:
    train_examples = pickle.load(f)

In [ ]:
#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('hfl/chinese-roberta-wwm-ext-large', device='cuda')
#Define your train examples. You need more than just two examples...
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=1000)

In [ ]:
model.save('D:\\hfl_pretraineds_document_0516')